# 🧠 Deep Learning–Based Abstractive Text Summarization
This Colab notebook trains a Transformer-based model (BART/T5) for abstractive summarization using the Hugging Face Transformers library.

In [9]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [5]:
!pip install -q transformers datasets evaluate rouge_score torch accelerate sentencepiece


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import evaluate



In [4]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
sample = dataset['train'][0]
print("ARTICLE:", sample['article'][:500])
print("\nSUMMARY:", sample['highlights'])


ARTICLE: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as s

SUMMARY: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .


In [8]:
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [9]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['article']
    targets = examples['highlights']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [15]:
from transformers import DataCollatorForSeq2Seq
import evaluate

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ROUGE metric
rouge = evaluate.load("rouge")

# Fixed compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in labels with pad token id
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in lab] for lab in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE and round results
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


In [11]:
!pip install -U transformers


In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',                # ✅ older version uses eval_strategy
    save_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
    report_to="none"                      # ✅ disables W&B logging
)


In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'].select(range(500)),  # small subset for testing
    eval_dataset=tokenized_datasets['validation'].select(range(100)),
    processing_class=tokenizer,        # replaces deprecated 'tokenizer' argument
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.769300,2.327293,0.238500,0.095800,0.193500,0.220000


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=500, training_loss=1.7693494873046876, metrics={'train_runtime': 8762.0142, 'train_samples_per_second': 0.228, 'train_steps_per_second': 0.057, 'total_flos': 609171975659520.0, 'train_loss': 1.7693494873046876, 'epoch': 1.0})

In [17]:
metrics = trainer.evaluate()
print(metrics)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
0,No log,2.304373,0.250000,0.101700,0.199700,0.228300


{'eval_loss': 2.304372549057007, 'eval_rouge1': 0.25, 'eval_rouge2': 0.1017, 'eval_rougeL': 0.1997, 'eval_rougeLsum': 0.2283}


In [19]:
text = """Artificial Intelligence is transforming industries by automating tasks and enabling new forms of creativity across domains such as healthcare, finance, and education."""

inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt").to(model.device)
summary_ids = model.generate(**inputs, max_length=100, num_beams=4, length_penalty=2.0)
print("SUMMARY:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


SUMMARY: Artificial Intelligence is transforming industries by automating tasks and enabling new forms of creativity across domains such as healthcare, finance, and education .


In [29]:
import gradio as gr
from transformers import BartTokenizer, BartForConditionalGeneration

# Load pretrained model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Abstractive summarization function
def summarize_text(text):
    # Preprocess the input text
    inputs = tokenizer(
        [text],
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    ).to(model.device)

    # Generate a truly abstractive summary
    summary_ids = model.generate(
        **inputs,
        max_length=200,           # Concise but informative
        min_length=60,            # Avoids trivial summaries
        num_beams=8,              # Improves quality
        do_sample=True,           # Enables creative paraphrasing
        top_k=50,                 # Sampling with top-k
        top_p=0.95,               # Nucleus sampling for diversity
        temperature=0.85,         # Slightly creative
        repetition_penalty=1.4,   # Avoids repeated phrases
        length_penalty=1.2,       # Encourages concise output
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # Optional: further clean any trailing incomplete sentences
    if summary.endswith(('.', '!', '?')) is False:
        summary += '.'
    return summary

# Save function
def save_summary(summary):
    with open("saved_summary.txt", "a") as f:
        f.write(summary + "\n\n")
    return "✅ Summary saved successfully!"

# Build Gradio UI
with gr.Blocks(theme="default") as demo:
    gr.Markdown(
        "<h2 style='text-align:center;'>🧠 Abstractive Summarization with Creative Rewriting (BART)</h2>"
    )

    with gr.Row():
        # Input Section
        with gr.Column(scale=1):
            text_input = gr.Textbox(
                label="Input Text",
                placeholder="Paste or type long text here...",
                lines=14,
                max_lines=30,
            )

            with gr.Row():
                clear_btn = gr.Button("🧹 Clear", variant="secondary")
                summarize_btn = gr.Button("🚀 Summarize", variant="primary")

        # Output Section
        with gr.Column(scale=1):
            summary_output = gr.Textbox(
                label="Generated Summary",
                lines=12,
                max_lines=25,
            )
            save_btn = gr.Button("💾 Save Summary", variant="secondary")

    # Button actions
    summarize_btn.click(summarize_text, inputs=text_input, outputs=summary_output)
    clear_btn.click(lambda: "", inputs=None, outputs=text_input)
    save_btn.click(save_summary, inputs=summary_output, outputs=summary_output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5d56cdc376430de79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
!git config --global user.name "RitikaShetty777"
!git config --global user.email "shettyritika636@gmail.com"


In [10]:
%cd /content/Abstractive-Summary



/content/Abstractive-Summary


In [15]:
!cp /content/Deep_Learning_Abstractive_Summarization.ipynb /content/Abstractive-Summary/


cp: cannot stat '/content/Deep_Learning_Abstractive_Summarization.ipynb': No such file or directory


In [16]:
!ls /content


Abstractive-Summary  sample_data
